# Tutorial on Basic Diffusion Analysis with GEMDAT

Welcome to this tutorial on diffusion analysis using the **GEMDAT** Python library for Molecular Dynamics (MD) simulations. GEMDAT is a powerful tool built on top of [Pymatgen](https://pymatgen.org/), which makes it straightforward to integrate into your existing Pymatgen-based workflows for materials analysis.

In this workshop, we'll introduce the basics of diffusion analysis with Gemdat. You'll learn how to:

- Import and process MD simulation data
- Perform diffusion analysis using GEMDAT
- Perform site analysis using GEMDAT
- Interpret and visualize the results

## How to Cite

When you use **GEMDAT code** in academic work **[please cite](https://github.com/GEMDAT-repos/GEMDAT/blob/main/CITATION.cff)**:
Azizi, V., Smeets, S., Lavrinenko, A. K., Ciarella, S., Famprikis, T., Landgraf, V., & Vasileiadis, A. GEMDAT (Version 1.6.0) [Computer software]. https://github.com/GEMDAT-repos/GEMDAT, doi: 10.5281/zenodo.8401669

### Loading Trajectory Data

In this section, we'll learn how to load trajectory data from a **VASP** simulations into GEMDAT and explore its basic properties. You can also use **LAMMPS** or **GROMACS** data.
For this tutorial, we provide VASP simulation data. However, if you're interested in using data from other MD software, feel free to use your own datasets or explore the [GEMDAT test data repository](https://github.com/GEMDAT-repos/test_data) for additional examples.

[GEMDAT Trajectory](https://gemdat.readthedocs.io/en/latest/api/gemdat_trajectory/#gemdat.trajectory.Trajectory) is an extension of Pymatgen's [Trajectory class](https://pymatgen.org/pymatgen.core.html#pymatgen.core.trajectory.Trajectory). It supports all the Pymatgen methods to create a trajectory, as well as additional methods to load data depending on your MD software.

Depending on the software used, you can load your simulation data as follows:

- **VASP**
  ```python
  Trajectory.from_vasprun('path/to/your/vasprun.xml')
  ```
- **LAMMPS**
  ```python
  Trajectory.from_lammps('path/to/coords_file', 'path/to/data_file', temperature=300, time_step=1)
  # Replace with your simulation temperature and time step in femtoseconds
  ```
- **GROMACS**
  ```python
  Trajectory.from_gromacs('path/to/topology_file', 'path/to/coords_file', temperature=300)
  # Replace with your simulation temperature
  ```

Let's proceed to load the trajectory data from a VASP simulation. We provide data for this workshop, but feel free to use your own data if available.

In [ ]:
# Import necessary libraries
import os
import numpy as np
from gemdat import Trajectory
from gemdat.metrics import TrajectoryMetricsStd
from gemdat.io import read_cif, write_cif
from gemdat.shape import ShapeAnalyzer


# This block will suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define the path to workshop folder
os.chdir('path/to/workshop/folder')

# Define the path to VASP data
VASPRUN = 'data/vasprun.xml'

# Read and cache the trajectory data
trajectory = Trajectory.from_vasprun(VASPRUN, cache='data/vasprun.xml.a45e320a.cache')

> **Note 1**: The code automatically caches the loaded data using pickle, saving time when rerunning.
>
> **Note 2**: If you are loading relaxation or NpT data, use the flag `constant_lattice=False` to allow for volume change.
>
> **Note 3**: For very large trajectories, you might want to limit the number of time steps or load a subset of the data to conserve memory using, e.g., `ionic_step_skip=10` or `ionic_step_offset=2500`.

### Exploring the Trajectory

Once the trajectory is loaded, let’s explore its basic properties. Printing the trajectory object provides a concise summary of its contents:

In [ ]:
# Print summary of the loaded trajectory
print(trajectory)

Here we will investigate AIMD simulation of $Na_3(Sb/W)S_4$ structure



You can also access specific attributes for more detailed information:

In [ ]:
# Retrieve and print key attributes of the trajectory

# Simulation temperature (K)
temperature = trajectory.metadata['temperature']
print(f"Simulation temperature: {temperature} K")

# Total simulation time in seconds
simulation_time = trajectory.total_time
print(f"Total simulation time: {simulation_time} s")

# Time step in picoseconds
time_step = trajectory.time_step_ps # or time_step = trajectory.time_step for seconds
print(f"Time step: {time_step} ps")

# Number of time steps
n_timesteps = len(trajectory)
print(f"Number of time steps: {n_timesteps}")

# Unique species in the simulation
species = trajectory.species
print(f"Unique species in the simulation: {set(species)}")

# Lattice of the simulation cell at time step 0
lattice = trajectory.get_lattice(0)
print(f"Volume of the simulation cell at time step 0: {lattice.volume:.2f} Å³")

# Structure at time step 0
structure = trajectory.get_structure(0)
print(f"Structure composition at Time Step 0: {structure.composition}")

# Coordinates (fractional)
coords = trajectory.coords  # or trajectory.positions
print(f"Shape of coordinates (time steps, atoms, dimensions): {coords.shape}")

You can slice the trajectory to exclude specific time steps. For example, to exclude the first 100 time steps:

In [ ]:
# Exclude the first 100 time steps from the trajectory
nr_first_steps = 500
print(f"Initial trajectory: {len(trajectory)} steps")
trajectory_sliced = trajectory[nr_first_steps:]
print(f"Trajectory after excluding the first {nr_first_steps} steps: {len(trajectory_sliced)} steps")

To focus on certain species, you can filter the trajectory. For instance, to extract a trajectory containing only antimony (Sb) and tungsten (W) atoms:

In [ ]:
# Filter the trajectory to include only Sb and W atoms
filtered_trajectory = trajectory.filter(['Sb', 'W'])
print(f"Filtered Trajectory Species: {set(filtered_trajectory.species)}")

You can also split the trajectory into multiple parts, which can be useful for analyzing separate segments:

In [ ]:
# Split the trajectory into 10 equal parts
list_of_trajectories = trajectory.split(n_parts=10, equal_parts=True)
print(f"Number of trajectories after split: {len(list_of_trajectories)}")

# Example: Access the first split trajectory
first_split = list_of_trajectories[0]
print(f"Length of the first split trajectory: {len(first_split)} steps")

> <div style="border: 1px solid #bce8f1; background-color: #d9edf7; padding: 10px; border-radius: 4px;">
>   <p style="margin: 0; font-weight: bold;">Task for You:</p>
>   <p>Save the structure at the last step of the simulation to a <code>.cif</code> file and visualize it with any preferred software.</p>
> </div>

### Analysing the Trajectory. Basic properties

[Gemdat `TrajectoryMetrics` class](https://gemdat.readthedocs.io/en/latest/api/gemdat_simulation_metrics/#gemdat.metrics.TrajectoryMetrics) computes various metrics from an MD simulation. We'll illustrate a few important ones.

First, we’ll filter the trajectory to focus on the diffusing atoms (Na in this case). For some of the upcoming analyses, only the trajectory of the diffusing species is necessary.

In [ ]:
# Extract the trajectory of diffusing atoms (Na in this case)
diff_trajectory = trajectory.filter('Na')

# Create a TrajectoryMetrics object for these Na atoms
diff_metrics = diff_trajectory.metrics()

We can calculate the particle density and concentration of the diffusing atoms:

In [ ]:
# Get concentration of diffusing atoms
particle_density = diff_metrics.particle_density()
concentration = diff_metrics.mol_per_liter()

print(f"Particle density of the diffusing element: {particle_density:.1e} atoms/m³")
print(f"Concentration of the diffusing element: {concentration:.2f} mol/L")

#### 1. Amplitude of Vibrations

Atomic vibrations in a crystal describe the motion of atoms around their (meta-)stable positions. From an MD simulation, positions of atoms at all time steps are known, allowing for the calculation of atomic displacements.

GEMDAT uses changes in the derivative of atomic displacement to identify vibrational motion. The vibrational amplitudes ($A$) are obtained by calculating the change in absolute displacement ($r_i$) of each atom ($i$) while the derivative of the displacement maintains the same sign (between time steps $t_a$ and $t_b$): $$ A = r_i(t_b) - r_i(t_a)$$

Fitting a Gaussian to the distribution of vibrational amplitudes yields an estimate for the average amplitude of atomic vibrations.


In [ ]:
# Uncomment %matplotlib if needed for interactive plotting (adjust according to your environment)
#%matplotlib notebook

In [ ]:
# Plot histogram of vibrational amplitudes with a Gaussian fit
fig = diff_trajectory.plot_vibrational_amplitudes(bins=50, n_parts = 5, backend='matplotlib')
fig.show()

# Calculate and print the average vibrational amplitude
avg_vibration_amplitude = diff_metrics.vibration_amplitude()
print(f"Average vibrational amplitude: {avg_vibration_amplitude:.3f} Å")

> **Note**:
> - GEMDAT has several built-in [plotting functions](https://gemdat.readthedocs.io/en/latest/plotting/#trajectory-and-displacements-plots). Plotly creates interactive plots, while matplotlib typically renders faster for larger datasets. By default, plotly is used, but you can switch to matplotlib using the `backend='matplotlib'` option.
>
> - To make matplotlib interactive, use environment-specific commands such as `%matplotlib notebook` (PyCharm), `%matplotlib widget` (VSCode), or `%matplotlib auto` (Spyder).

> <div style="border: 1px solid #bce8f1; background-color: #d9edf7; padding: 10px; border-radius: 4px;">
>   <p style="margin: 0; font-weight: bold;">Task for You:</p>
>   <li>1. Enable interactive matplotlib plots and rerun the relevant cells.</li>
>   <li>2. Try running the same plots with the <code>plotly</code> backend to see the difference.</li>
> </div>

#### 2. Attempt Frequency

We can compute a vibration frequency spectrum via a Fourier transform of atomic displacement derivatives. The vibrational spectrum is obtained by computing the derivative of the absolute displacement ($\Delta r_i$) per atom ($i$) at every time step ($t$): $$\Delta r_i(t)=r_i(t) - r_i(t-1)$$
The vibration frequency (also known as the attempt frequency, ($\nu^*$) can then be derived from the Fourier transformation of this spectrum. This frequency is often used in rate theory to estimate jump rates.

> **Note**: This approach is distinct from a phonon spectrum, since it examines vibrations of individual atoms rather than collective modes.

In [ ]:
# Plot the vibration frequency spectrum of Na ions
fig = diff_trajectory.plot_frequency_vs_occurence(backend='matplotlib')
fig.show()

# Calculate the attempt frequency and its std
attempt_frequency, attempt_frequency_std = diff_metrics.attempt_frequency()
print(f"Attempt frequency: ({attempt_frequency:.2e} ± {attempt_frequency_std:.2e}) Hz")

#### 3. Displacements

The mean squared displacement (MSD) measures how far, on average, particles move from their initial positions over time. Several definitions of MSD are commonly used:
* **Direct**: $$ MSD(t) = \frac{1}{N} \sum_i^N \left(r_i(t)-r_i(0)\right)^2 ,$$ which simply computes how much particles have moved from their initial positions after time $t$.
* **Multi-start direct**: $$ MSD(t) = \frac{1}{N * [\mathcal{T}]} \sum_{t_0 \in \mathcal{T}} \sum_i^N \left(r_i(t+t_0)-r_i(t_0)\right)^2 ,$$ which averages the direct method over different starting times $t_0$ in a finite set $\mathcal{T}={0, 10s, 20s, \cdots}$, improving statistical accuracy. This method is $[\mathcal{T}]$ times as slow as the direct one, but it can be parallelized for optimal performance.
* **Window method**: $$ MSD(t) = \frac{1}{N}  \sum_i^N \frac{1}{N-t} \sum_{s=0}^{N-t-1} \left(r_i(t+s)-r_i(s)\right)^2 ,$$ which extends the multi-start method to include all possible time windows of size $t$. This method is more computationally demanding than the others, but it can be formulated using FFT making it very efficient, while retaining its superior precision.

GEMDAT implements the **window method** using **FFT** (from [Calandrini et al. (2011)](https://www.neutron-sciences.org/articles/sfn/abs/2011/01/sfn201112010/sfn201112010.html) for efficient calculation.

Let’s compute and visualize the MSD of the diffusing atoms. We’ll start by calculating the MSD for Na and plotting the MSD evolution over time.

In [ ]:
# Calculate the mean squared displacement for diffusing atoms
msd_data = diff_trajectory.mean_squared_displacement()

# Extract the MSD at the last time step
msd_last_time_step = np.mean(msd_data[:, -1])

print(f"Na-ion MSD after {simulation_time / 1e-12:.0f} ps AIMD ({temperature:.0f} K): {msd_last_time_step:.2f} Å²")

# Plot MSD evolution over time for each element in the structure
fig = trajectory.plot_msd_per_element()
fig.show()

Besides the MSD, GEMDAT can also visualize displacements for each element or each individual atom.

In [ ]:
# Plot displacement evolution over time for each element
fig = trajectory.plot_displacement_per_element(backend='matplotlib')
fig.show()

# Plot displacement evolution for each diffusing atom
fig = diff_trajectory.plot_displacement_per_atom(backend='matplotlib')
fig.show()

# Plot histogram of final displacements of diffusing atoms
fig = diff_trajectory.plot_displacement_histogram(backend='matplotlib')
fig.show()


#### 4. Diffusion Properties

From the MSD, you can estimate the **tracer diffusivity** ($D^*$): $$D^*=\frac{1}{2dNt} \sum_{i=1}^N \left(r_i(t+t_0)-r_i(t_0)\right)^2 ,$$
where $r_i(t+t_0)$ is the displacement of a single atom with respect to the starting position $r_i(t_0)$, $t$ the simulated time, $N$ the number of diffusing atoms, and $d$ the number of diffusion dimensions.

GEMDAT offers a direct calculation of tracer diffusivity, followed by **ionic conductivity** ($σ$) via the Nernst−Einstein relation:
$$σ=\frac{ne^2z^2}{k_bT} D^*$$
where $n$ is the diffusing particle density, $e$ the elementary electron charge, $z$ the ionic charge, $k_B$ Boltzmann’s constant, and $T$ the temperature in Kelvin.

Let’s calculate the tracer diffusivity and conductivity for Na atoms.

In [ ]:
# Calculate tracer diffusivity for Na (assuming 3D diffusion)
tracer_diffusivity = diff_metrics.tracer_diffusivity(dimensions=3)
print(f"Tracer diffusivity of Na-ions at {temperature:.0f} K: {tracer_diffusivity:.2e} m²/s")

# Calculate tracer conductivity for Na
tracer_conductivity = diff_metrics.tracer_conductivity(z_ion=1, dimensions=3)
print(f"Tracer conductivity of Na-ions at {temperature:.0f} K: {tracer_conductivity:.2f} S/m")

The [Gemdat `TrajectoryMetricsStd` class](https://gemdat.readthedocs.io/en/latest/api/gemdat_simulation_metrics/#gemdat.metrics.TrajectoryMetricsStd) calculates statistical errors of a trajectory list. Below, we calculate mean values from the original trajectory, divided into 10 parts.

In [ ]:
# Split the diffusing atom trajectory into 10 equal parts
list_of_trajectories = diff_trajectory.split(n_parts=10, equal_parts=True)

# Create a TrajectoryMetricsStd object
diff_metrics_std = TrajectoryMetricsStd(trajectories=list_of_trajectories)

# Calculate average vibrational amplitude with standard deviation
avg_vib_amp = diff_metrics_std.vibration_amplitude()
print(f"Average vibrational amplitude: {avg_vib_amp:.3f} Å")

# Calculate tracer diffusivity with standard deviation
avg_tracer_diffusivity = diff_metrics_std.tracer_diffusivity(dimensions=3)
print(f"Tracer diffusivity of Na-ions at {temperature:.0f} K: {avg_tracer_diffusivity:.2e} m²/s")

# Calculate tracer conductivity with standard deviation
avg_tracer_conductivity = diff_metrics_std.tracer_conductivity(z_ion=1, dimensions=3)
print(f"Tracer conductivity of Na-ions at {temperature:.0f} K: {avg_tracer_conductivity:.2f} S/m")

#### 5. Radial Distribution Function

The Radial Distribution Function (RDF) reveals the density of an element versus distance from another element during the MD simulation. The RDF is determined by calculating the distance between all species pairs for each time step and binning them into a histogram. The histogram is then normalized, where normalization is the number density of the system multiplied by the volume of the spherical shell.

Below, we calculate the RDF of Na relative to Sb and W:

In [ ]:
# Calculate the RDF between Na and [Sb, W]
rdf_cations = trajectory.radial_distribution_between_species(
    specie_1='Na', specie_2=['Sb','W'], resolution=0.1, max_dist=10)

# Plot the RDF
fig = rdf_cations.plot()
fig.show()

### Analysing the Trajectory. Site-specific properties

In crystalline ionic conductors, diffusion often occurs via transitions between (meta-)stable sites. If the crystallographic positions of the diffusing atoms are known, you can incorporate them in the analysis.

Below, we demonstrate how to load a known structure containing these sites (or generate them from the MD data) and carry out further analysis.

#### 1. Loading structure with crystallographic sites

There are multiple ways to load such a structure:

1) Create a structure with Pymatgen [Structure](https://pymatgen.org/pymatgen.core.html#pymatgen.core.structure.Structure) or other compatible libraries

2) Import a structure from the [gemdat known materials library](https://gemdat.readthedocs.io/en/latest/api/gemdat_io/#gemdat.io.get_list_of_known_materials), which includes:

* Argyrodite
* Lithium aluminium germanium phosphate (LAGP)
* Lithium aluminum titanium phosphate (LATP)
* $Li_3PS_4$
* $Li_{10}SnP_2S_{12}$
* $lambda-NaMnO_2$
* $Na_3PS_4$

3) Load from a CIF file
4) Extract from the MD trajectory itself

Here's an example of loading a structure from a **CIF file**:

In [ ]:
# Read a CIF file
custom_structure = read_cif('data/structure_1.cif')
custom_structure

> **Note**:
> 1. If your floating species (e.g., Na) occupies partially disordered sites, **include all possible positions in your reference structure with full occupancy to represent potential sites**.
> 2. If the structure includes other atoms (e.g., W, Sb, or S) that will not be occupied by the floating species, **consider removing those sites to avoid confusion**.

In [ ]:
# Example: Adjusting the loaded structure
custom_structure = read_cif('data/structure_1.cif')
custom_structure.make_supercell((2, 2, 2)) # Make supercell

# Remove other species besides the floating specie (Na)
custom_structure.remove_species(['S','Sb','W'])

# Replace partial occupancies with full occupancy for the floating species
for idx,site in enumerate(custom_structure.sites):
    custom_structure.replace(
        idx=idx,
        species=site.species.elements[0],
        label=site.label
    )

Finding jump sites **from an MD Trajectory**

If you do not have a reference structure of the floating species, you can extract stable sites directly from the MD simulation:

1. Filter the trajectory by the diffusing atom.
2. Convert the trajectory to a 3D density volume.
3. Find peaks in that volume.
4. Convert the peaks into a Pymatgen structure.

Let’s demonstrate steps (2) to (4). We’ve already filtered the trajectory for Na as `diff_trajectory`.

First, convert `diff_trajectory` to a `Volume` object. A volume in this context is a discretized grid version of the trajectory. Each 3D voxel bins the coordinates over the simulation. The resolution is in Å and determines the voxel size.

The resolution is the minimum size of the voxels in Ångstrom.

> Note: A voxel does not necessarily represent a cubic or even orthorhombic volume, and depends on the lattice parameters of the trajectory.

In [ ]:
# Convert diff_trajectory to a 3D volume with a specified resolution
vol = diff_trajectory.to_volume(resolution=0.4)

Next, convert the volume to a structure of extracted sites using a [watershed algorithm](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_watershed.html). Increase the `background_level` if too many spurious peaks are found. Sometimes the generated sites need to be cleaned up a bit manually.

In [ ]:
# Convert volume to structure (peaks) with Na
sites = vol.to_structure(specie='Na', background_level=0.6)
print('Number of sites extracted from the MD trajectory: ',len(sites.sites))
print('Number of sites loaded from custom structure from .cif file: ',len(custom_structure.sites))

In [ ]:
# Optionally, save the extracted sites to a CIF file
write_cif(sites,'extracted.cif')

> <div style="border: 1px solid #bce8f1; background-color: #d9edf7; padding: 10px; border-radius: 4px;">
>   <p style="margin: 0; font-weight: bold;">Task for You:</p>
>   <li>Adjust <code>resolution</code> and <code>background_level</code> to match or approximate your custom structure. Compare the extracted result with your reference <code>.cif</code> in a visualization tool like VESTA.</li>
> </div>

#### 2. Visualising probability density of diffusion ion

We can visualize the density plot from our volume data, overlaying a reference structure. The density can be saved to an interactive HTML or to a `.vasp` file.

In [ ]:
# Load another structure from CIF with multiple site types
custom_structure_2 = read_cif('data/structure_2.cif')
custom_structure_2.make_supercell((2, 2, 2))
custom_structure_2.remove_species(['S','Sb','W'])
for idx,site in enumerate(custom_structure_2.sites):
    custom_structure_2.replace(
        idx=idx,
        species=site.species.elements[0],
        label=site.label
    )

In [ ]:
# Plot 3D density and overlay custom_structure_2
fig = vol.plot_3d(structure=custom_structure_2)
config = {
        'toImageButtonOptions': {
            'format': 'png',  # one of png, svg, jpeg, webp
            'transparent': True,
            'filename': 'custom_image',
            'scale': 10  # Multiply title/legend/axis/canvas sizes by this factor
        }
    }

# Save density plot to interactive HTML file
fig.write_html('density_plot.html',config=config)
fig.show()

To save the density to a `.vasp` file (for inspection in external tools):

In [ ]:
# Save the volume data in a VASP-compatible format
vasp_output = vol.to_vasp_volume(sites, filename='density_plot.vasp')

#### 3. Finding transition events

With the reference site structure in hand, you can identify how often atoms jump between specific sites. By defining a `site_radius` for each crystallographic site, GEMDAT determines when an atom is considered to be occupying a site. If a `site_radius` is undefined the value of doubled vibrational amplitude will be used and the code automatically avoids double-counting if two sites overlap by reducing to half the distance between the two sites, thus preventing overlapping sites.

Here, we load a structure that has two types of Na sites, then detect transitions.

In [ ]:
# Load and prepare a structure with two site types for Na
custom_structure_2 = read_cif('data/structure_2.cif')
custom_structure_2.make_supercell((2, 2, 2))
custom_structure_2.remove_species(['S','Sb','W'])
for idx,site in enumerate(custom_structure_2.sites):
    custom_structure_2.replace(
        idx=idx,
        species=site.species.elements[0],
        label=site.label
    )

# Create transitions object
transitions = trajectory.transitions_between_sites(
    sites=custom_structure_2,
    floating_specie='Na',
    site_radius={'Na1':1.5, 'Na2':1.5}
)

The transitions are collected in a DataFrame, which tracks the atom index, start and destination site indices, and time step of the jump.

> **Note**: An index of `-1` means the atom is not assigned to any site.

In [ ]:
# View the transition events
transitions.events

You can further compute the radial distribution (RDF) specific to each site or site transition.

> **Note**: RDF naming convention in GEMDAT:
> - "@site_name": the RDF when the atom is assigned to a specific site.
> - "site_A->site_B": the RDF when the atom jumps from site A to site B.
> - "~>site_name": for failed jumps where the atom leaves and returns to the same site.

In [ ]:
# Compute the site-specific RDF
rdf_data = transitions.radial_distribution(
    floating_specie='Na',
)

# Plot the results
for rdfs in rdf_data.values():
    fig = rdfs.plot(backend='matplotlib')
    fig.show()

You can also get the **atom locations** (fraction of Na atoms occupying each site type) and the **occupancy** (fraction of sites occupied by Na atoms) over time.

In [ ]:
# Calculate atom locations per site type
atom_locations = transitions.atom_locations()
print(f'Atom locations per site type: "Na1":{atom_locations["Na1"]:.2f}, "Na2":{atom_locations["Na2"]:.2f}')

# Calculate average occupancy per site type
average_occupancy = transitions.occupancy_by_site_type()
print(f'Average occupancy per site type: "Na1":{average_occupancy["Na1"]:.2f}, "Na2":{average_occupancy["Na2"]:.2f}')

# Retrieve occupancy for each individual site
occupancy = transitions.occupancy()
occupancy

> **Note**:
> - Atom locations differ from occupancy:
> $$atom\_locations = \frac{1}{n_{atoms}} \sum_{i}^{n_{sites^i}} \sum_{k}^{n_{atoms}} \frac{t_{occupied}}{t_{total}}$$
> $$average\_occupancy = \frac{1}{n_{sites^i}} \sum_{i}^{n_{sites^i}} \sum_{k}^{n_{atoms}} \frac{t_{occupied}}{t_{total}}$$
> where $n_{atoms}$ - number of floating atoms; $n_{sites^i}$ - number of sites of type $i$; $t_{occupied}$ - number of timesteps when atom $k$ occupies site $i$; $t_{total}$ - total number of timesteps.
>
> - The output of occupancy per each single site is Structure object with occupancies (fraction of time when site is occupied by floating atoms) set on each site.

> <div style="border: 1px solid #bce8f1; background-color: #d9edf7; padding: 10px; border-radius: 4px;">
>   <p style="margin: 0; font-weight: bold;">Task for You:</p>
>   <li>Experiment with various <code>site_radius</code> values in <code>transitions_between_sites</code> function. Calculate and compare the resulting RDFs, atom locations, and occupancy.</li>
> </div>

#### 4. Analysis of jumps

When crystallographic sites are known, it’s straightforward to detect transitions between them. The jumps data is stored in a DataFrame, including the atom index, start and destination site indices, and time steps of the jump.

> **Note**
> To avoid overcounting vibrations at the site boundary as transitions, GEMDAT offers parameters like `site_inner_fraction` (distance threshold) and `minimal_residence` (time threshold).
> 1) The **distance threshold**. With specified radius of inner sphere, the jump will be counted when the atom leaves the inner sphere of starting site and enters the inner sphere of the destination site.
>
> 2) The **time threshold**. When the atom arrives to destination site it should spend several time steps at the destination site before jump is counted.

Below, we illustrate how to use these parameters.

In [ ]:
transitions = trajectory.transitions_between_sites(
    sites=custom_structure_2,
    floating_specie='Na',
    site_radius={'Na1':1.5, 'Na2':1.5},
    site_inner_fraction=1.0, # The fraction of site_radius considered "inner"
)

# minimal_residence=0 means no time threshold is required once the atom arrives
jumps = transitions.jumps(minimal_residence=0)
jumps.data

We can visualize statistics of jump distances and occurrences over time.

In [ ]:
# Plot jumps vs distance and jumps vs time
fig = jumps.plot_jumps_vs_distance(backend='matplotlib')
fig.show()
fig = jumps.plot_jumps_vs_time(backend='matplotlib')
fig.show()

We can also depict the jumps as lines between sites in 3D, with the line width reflecting how many jumps occurred.

In [ ]:
jumps.plot_jumps_3d()

From the jump data, you can count the **number of jumps** ($J_i$) and get the **mean jump rate** ($Γ_i$) using
$$Γ_i = \frac{J_i}{Nt}$$
where $N$ is the number of diffusing atoms, the subscript $i$ defines a type of jump, and $t$ is the simulation time.

To get an estimate of the uncertainty in the jump rate the MD simulation is divided into ten different parts (this parameter can be adjusted).

In [ ]:
# Total number of jumps
number_jumps = jumps.n_jumps
print(f"Total number of jumps: {number_jumps}")

# Mean jump rates (per jump type), with std if multiple parts are used
jump_rates = jumps.rates(n_parts=5)
print(f"\nMean jump rates (total jumps / second) with std per jump type: \n",round(jump_rates,2))

You can also calculate the **jump rate diffusivity** ($D_J$) via the Einstein–Smulochowski relation:
$$D_J = \sum_{i} \frac{Γ_ia_i^2}{2d}$$
where $i$ are the different types of jumps, $a_i$ is the jump distance of jump type $i$, and $d$ the number of diffusion dimensions. The sum is over all types of jumps, since in most solid electrolytes several different types of jumps contribute to macroscopic diffusion.

In [ ]:
jump_diffusivity = jumps.jump_diffusivity(dimensions=3)
print(f"Average jump diffusivity: {jump_diffusivity:.2e} m²/s")

If you run MD simulations at multiple temperatures, you can fit an Arrhenius equation to find the activation energy. Alternatively, you can compute the **activation energy** ($ΔE_i^A$) for a jump type ($i$) at a given temperature:
$$ΔE_i^A = -k_B T ln(\frac{Γ_i^{eff}}{ν^*})$$
where $k_B$ is Boltzmann’s constant, $T$ the temperature in Kelvin, $Γ_i^{eff}$ the effective jump frequency, and $ν^*$ the attempt frequency.

The activation energy can be seen as a measure for the jump probability: if the activation energy is low, the jump probability is high, and vice versa.

A jump from site A to B can have a different energy barrier as the reverse jump due to the difference in site energy, even though the number of A−B and B−A jumps will be the same in equilibrium. The amount of time an atom spends at a certain site should be taken into account to correctly determine the activation energy. The effective jump rate, $Γ_i^{eff}$, thus differs from the jump rate by taking into account the fraction of time that the diffusing atoms occupy a type of site ($o_j$):
$$Γ_i^{eff} = \frac{J_i}{tNo_j}$$
where $t$ is the total simulated time, and $N$ the number of diffusing atoms.

By incorporating site occupancy the sites with lower occupancy will have lower activation energies, correctly representing the difference in activation energy between A−B and B−A jumps.

In [ ]:
act_energy = jumps.activation_energies(n_parts=5)
print(f"Mean activation energy (eV) per jump type:\n{act_energy}")

If jumps occur in quick succession within a chosen radius and timescale, they can be classified as **collective**. GEMDAT identifies these by checking whether multiple jumps happen within one vibration period ($\frac{1}{ν^*}$ seconds) and within some spatial threshold.

In [ ]:
# Number of solo (i.e., not collective) jumps
solo_jumps = jumps.n_solo_jumps
frac_solo_jumps = jumps.solo_fraction
print(f"Number of solo (not collective) jumps: {solo_jumps}, "
      f"fraction of solo jumps = {frac_solo_jumps:.2f}")

In [ ]:
# Plot correlated jumps
fig = jumps.plot_collective_jumps(backend='matplotlib')
fig.show()

> <div style="border: 1px solid #bce8f1; background-color: #d9edf7; padding: 10px; border-radius: 4px;">
>   <p style="margin: 0; font-weight: bold;">Task for You:</p>
>   <li>Experiment with various <code>site_radius</code> and <code>site_inner_fraction</code> values in <code>transitions_between_sites</code> function and <code>minimal_residence</code> values in <code>transitions.jumps</code> function. Calculate and compare the resulting jump rates, activation energies, or jump diffusivities. Consider changing <code>n_parts</code> parameter.</li>
> </div>

#### 5. Shape analysis

GEMDAT provides a generalized shape analysis algorithm (`ShapeAnalyzer`) for symmetrical clusters. You supply:
1. A symmetrized structure with unique sites.
2. A trajectory in P1 (supercell) format.

The algorithm folds the trajectory back onto the asymmetric unit and gathers localized positions. This is useful for cluster-based shape analysis, plots, and more.

Below, we demonstrate constructing the `ShapeAnalyzer` from a structure.
>
> **Note:**
> Do **not** apply supercell to loaded structure, it will be applied later.


In [ ]:
# Adjusting the loaded structure, do not apply supercell
custom_structure_1 = read_cif('data/structure_1.cif')

# Remove other species besides the floating specie (Na)
custom_structure_1.remove_species(['S','Sb','W'])

# Replace partial occupancies with full occupancy for the floating species
for idx,site in enumerate(custom_structure_1.sites):
    custom_structure_1 = custom_structure_1.replace(
        idx=idx,
        species=site.species.elements[0],
        label=site.label
    )

# Create the shape analyzer by symmetrizing the input structure
sa = ShapeAnalyzer.from_structure(custom_structure_1)

Next, we run shape analysis by specifying a radius (in Å) that determines which trajectory points belong to the cluster.
The supercell is used to fold the trajectory coordinates to the same lattice as the input sites. A warning will be raised if the lattices are not close. Alternatively, you can pass any numpy array with fractional coordinates to `sa.analyze_positions()`

In [ ]:
supercell = (2, 2, 2)
radius = 1.6  # Å

# Analyze the trajectory around each site in the asymmetry
shapes = sa.analyze_trajectory(
    trajectory=diff_trajectory,
    supercell=supercell,
    radius=radius,
)

shapes[0]

We can plot the resulting shape data. Each shape is a set of coordinates (centered on a particular site) from the entire simulation. You can include nearby sites on the plot for context by using `sites` parameter.

In [ ]:
bins = np.linspace(-radius, radius, 100)

# Plot the 2D histograms (XY, XZ, YZ) of positions around each unique site
for shape in shapes:
    sites = custom_structure_1.get_sites_in_sphere(pt=shape.origin, r=radius)
    fig = shape.plot(bins=bins, sites=sites, backend='matplotlib')
    fig.show()

> <div style="border: 1px solid #bce8f1; background-color: #d9edf7; padding: 10px; border-radius: 4px;">
>   <p style="margin: 0; font-weight: bold;">Task for You:</p>
>   <li>Run <code>ShapeAnalyzer</code> and plot shapes for each unique site loading <code>structure_2.cif</code>.</li>
> </div>

You can **optimize site positions** using the `ShapeAnalyzer.optimize_sites()` method, which aligns site centroids to the center of mass of the shape.

In [ ]:
sa_shifted = sa.optimize_sites(shapes=shapes)
sa_shifted

Alternatively, you can shift sites by a custom vector array.

In [ ]:
vectors = (
    [1, 0, 0],
    [2, 0, 0],
    [3, 0, 0],
)

sa_shifted = sa.shift_sites(vectors=vectors)
sa_shifted

Finally, you can retrieve a standard Pymatgen `Structure` object back from the shape analyzer.

In [ ]:
sa_shifted = sa_shifted.to_structure()
sa_shifted

## Conclusion

This tutorial demonstrated several workflows and methods provided by GEMDAT for analyzing MD simulation data:

1. Loading and processing MD trajectories.
2. Calculating vibrational amplitudes, attempt frequencies, MSD, diffusivity, and ionic conductivity.
3. Extracting and analyzing site transitions and jump statistics.
4. Visualizing probability density and performing site-shape analysis.

We hope this guide helps you integrate GEMDAT into your own MD data workflows.

P.S. If you're trying GEMDAT with your data, your experience, questions, bugs you encountered, and suggestions for improvement are important to the success of the project. Feel free to open a new [issue here](https://github.com/GEMDAT-repos/GEMDAT/issues) to ask a question, suggest improvements/new features, or report any bugs that you ran into.